## Importing libraries

In [1]:
!pip install unsloth

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.0/59.0 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.6/195.6 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 72.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 71.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import os
os.environ["HF_TOKEN"] = input("Enter your Hugging Face Token: ")


In [3]:
## Large langugae model uses the 32 bits to store the model by using qunatization they reduce them to 4 or 8 bit qunatization
from unsloth import FastLanguageModel ## A very good library around the hugging face that helps to load model by using often in reduced precision (e.g., 4-bit quantization).
max_seq_length = 2048 ## Maximum sequence for the tokenize input
dtype = None ## Data type of the model weights use the default precision
load_in_4bit = True ## Enable 4 bit qunatization

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="deepseek-ai/DeepSeek-R1-Distill-Llama-8B", ## Pretrained Model from the hugging face
    max_seq_length=max_seq_length,## Set Maximum as set
    dtype=dtype,## Use default as it is set to none
    load_in_4bit=load_in_4bit,## Use 4 bit quantization to load the model
    token=os.getenv("HF_TOKEN"),  # add your HF token
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.3.15: Fast Llama patching. Transformers: 4.48.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.96G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/236 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/53.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [4]:
prompt_style = """Below is an instruction that describes a task, paired with an input that provides further context.
Write a response that appropriately completes the request.
Before answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.

### Instruction:
You are a math expert with advanced knowledge in mathematical reasoning, problem-solving, and theoretical concepts.
Please answer the following math question. You have to Give the Answer and solve The Response is like how to give the Answer
like this 'Response': '🍎 = 4' for example

### Question:
{}

### Response:
<think>{}"""

In [5]:
from datasets import load_dataset
from transformers import AutoTokenizer

# Load dataset from CSV
dataset = load_dataset("csv", data_files="/content/emoji_math.csv")

# Initialize tokenizer (replace with your model name)

EOS_TOKEN = tokenizer.eos_token  # Must add EOS_TOKEN

# Define your training prompt style
train_prompt_style = """Below is an instruction that describes a task, paired with an input that provides further context.
Write a response that appropriately completes the request.
Before answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.

### Instruction:
You are a math expert with advanced knowledge in mathematical reasoning, problem-solving, and theoretical concepts.
Please answer the following math question. You have to Give the Answer and solve The Response is like how to give the Answer
like this 'Response': '🍎 = 4' for example

### Question:
{}

### Response:
<think>
{}
</think>
{}""" + EOS_TOKEN

# Formatting function
def formatting_prompts_func(examples):
    inputs = examples["Question"]
    cots = examples["Complex_CoT"]
    outputs = examples["Response"]
    texts = []
    for input, cot, output in zip(inputs, cots, outputs):
        text = train_prompt_style.format(input, cot, output)
        texts.append(text)
    return {"text": texts}

# Apply formatting function
dataset = dataset.map(formatting_prompts_func, batched=True)

# Check formatted data
print(dataset["train"][0])


Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/30 [00:00<?, ? examples/s]

{'Question': '🍎 + 🍎 + 🍎 = 12', 'Complex_CoT': 'Break it down: 🍎 + 🍎 + 🍎 = 12 → Divide both sides by 3 → 🍎 = 4', 'Response': '🍎 = 4', 'text': "Below is an instruction that describes a task, paired with an input that provides further context.\nWrite a response that appropriately completes the request.\nBefore answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.\n\n### Instruction:\nYou are a math expert with advanced knowledge in mathematical reasoning, problem-solving, and theoretical concepts.\nPlease answer the following math question. You have to Give the Answer and solve The Response is like how to give the Answer\nlike this 'Response': '🍎 = 4' for example\n\n### Question:\n🍎 + 🍎 + 🍎 = 12\n\n### Response:\n<think>\nBreak it down: 🍎 + 🍎 + 🍎 = 12 → Divide both sides by 3 → 🍎 = 4\n</think>\n🍎 = 4<｜end▁of▁sentence｜>"}


In [6]:
model = FastLanguageModel.get_peft_model(
    model,
    r=8,  # Lower rank for faster updates
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
    lora_alpha=8,  # Lower influence speeds up training
    lora_dropout=0.1,  # Small dropout to improve generalization
    bias="none",
    use_gradient_checkpointing=False,  # Turn off for faster training
    random_state=3407,
    use_rslora=False,
    loftq_config=None,
)

from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset['train'],
    dataset_text_field="text",
    max_seq_length=256,  # Reduce sequence length for faster training
    dataset_num_proc=1,  # Reduce CPU overhead
    args=TrainingArguments(
        per_device_train_batch_size=4,  # Increase batch size if GPU allows
        gradient_accumulation_steps=2,  # Reduce accumulation to speed up steps
        warmup_steps=1,  # Reduce warmup
        max_steps=3,  # Reduce total steps for quick testing
        learning_rate=5e-4,  # Slightly higher learning rate for faster convergence
        fp16=True,  # Force fp16 if supported
        bf16=False,  # Disable bf16 for stability
        logging_steps=1,  # Log more frequently for monitoring
        optim="adamw_torch",  # Faster optimizer
        weight_decay=0.01,
        lr_scheduler_type="cosine",  # Cosine scheduler adapts better
        seed=3407,
        output_dir="outputs",
        report_to="none"
    ),
)


Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.1.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.
Unsloth 2025.3.15 patched 32 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


Unsloth: Tokenizing ["text"]:   0%|          | 0/30 [00:00<?, ? examples/s]

In [7]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 30 | Num Epochs = 1 | Total steps = 3
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 2
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 2 x 1) = 8
 "-____-"     Trainable parameters = 20,971,520/8,000,000,000 (0.26% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,3.399800
2,3.420900
3,2.838600


In [8]:
question = "🍎 + 🍎 + 🍎 + 🍎 + 🍎= 20"


FastLanguageModel.for_inference(model)  # Unsloth has 2x faster inference!
inputs = tokenizer([prompt_style.format(question, "")], return_tensors="pt").to("cuda")

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1200,
    use_cache=True,
)
response = tokenizer.batch_decode(outputs)
print(response[0].split("### Response:")[1])


<think>
Okay, so I've got this math problem here: 🍎 + 🍎 + 🍎 + 🍎 + 🍎 = 20. Hmm, I need to figure out what each 🍎 equals. Let me break it down step by step.

First, I notice that there are five 🍎s being added together. So, if I let one 🍎 be equal to some number, say 'x', then adding five of them together would be 5 times x. That makes sense because 5🍎 is the same as 5x.

So, the equation becomes 5x = 20. Now, to find the value of x, I need to solve for it. To isolate x, I can divide both sides of the equation by 5. That way, I'm getting x by itself.

Let me do the division: 20 divided by 5 is 4. So, x equals 4. That means each 🍎 is worth 4.

Wait, let me double-check to make sure I didn't make a mistake. If each 🍎 is 4, then five of them would be 5 times 4, which is 20. Yep, that matches the original equation. So, my answer makes sense.

I guess another way to think about it is by using addition. Adding one 🍎 five times: 4 + 4 + 4 + 4 + 4. Let's add them step by step. 4 plus 4 is 8, plu

In [9]:
new_model_local = "DeepSeek-R1-Medical-COT"
model.save_pretrained(new_model_local)
tokenizer.save_pretrained(new_model_local)

model.save_pretrained_merged(new_model_local, tokenizer, save_method = "merged_16bit",)

Unsloth: You have 1 CPUs. Using `safe_serialization` is 10x slower.
We shall switch to Pytorch saving, which might take 3 minutes and not 30 minutes.
To force `safe_serialization`, set it to `None` instead.
Unsloth: Kaggle/Colab has limited disk space. We need to delete the downloaded
model which will save 4-16GB of disk space, allowing you to save on Kaggle/Colab.
Unsloth: Will remove a cached repo with size 6.0G


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 5.22 out of 12.67 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


 34%|███▍      | 11/32 [00:00<00:01, 15.43it/s]
We will save to Disk and not RAM now.
100%|██████████| 32/32 [02:55<00:00,  5.47s/it]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving DeepSeek-R1-Medical-COT/pytorch_model-00001-of-00004.bin...
Unsloth: Saving DeepSeek-R1-Medical-COT/pytorch_model-00002-of-00004.bin...
Unsloth: Saving DeepSeek-R1-Medical-COT/pytorch_model-00003-of-00004.bin...
Unsloth: Saving DeepSeek-R1-Medical-COT/pytorch_model-00004-of-00004.bin...
Done.
